# Coursera Capstone project

In [798]:
import pandas as pd
import numpy as np
import requests
from lxml import html
import geocoder

-----------------------------------------------------------------------------
# Scraping Wikipedia for data

Instead of making multiple scraping request its better to fetch whole table at once, find a pattern and clean the text.

In [799]:
#--------------------------Making request and fetching data form wiki
pageContent=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
tree = html.fromstring(pageContent.content)
table_data = tree.xpath('/html/body/div[3]/div[3]/div[4]/div/table[1]')
table_data = table_data[0].text_content()

--------------------------Pre-porcessing fetched data--------------------------

In [800]:
table_data=table_data.split('\n\n\n')
table_data.pop(0)

postal_code=[]
borough=[]
neig_hood=[]

for x in table_data:
    y=x.split('\n')
    if y[1]=='Not assigned':
        pass
    else:
        postal_code.append(y[0])
        borough.append(y[1])
        if y[2]=='Not assigned':
            neig_hood.append(y[1])
        else:
            neig_hood.append(y[2])

--------------------------Conversion to DataFrame--------------------------

In [801]:
data_framee = pd.DataFrame(
    {'Postcode': postal_code,
     'Borough': borough,
     'Neighbourhood': neig_hood
    })
data_framee

#----------------------------------Merging both dataframes on postalcodes
Geospatial_Coordinates = pd.read_csv('/Users/aayushbhargava/Downloads/Geospatial_Coordinates.csv')
data_framee2 = pd.merge(left=data_framee,right=Geospatial_Coordinates, left_on='Postcode', right_on='Postal Code')
data_framee2=data_framee2.drop(columns='Postal Code')
print (data_framee2.shape)
data_framee2.head()

(210, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


In [802]:
#--------------------------Cleaning Data frame
d = data_framee.groupby('Postcode', as_index=False).agg({'Borough':'unique','Neighbourhood':list})
d.sort_values(by=['Postcode'])
d['Neighbourhood']=d['Neighbourhood'].apply(','.join)
d['Borough']=d['Borough'].apply(''.join)
d

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


GeoCoder is not Working

'''
lat_lng_coords=None
while(lat_lng_coords is None):
    g = geocoder.google('M9M, Toronto, Ontario')
    lat_lng_coords = g.latlng
lat_lng_coords
print (lat_lng_coords[0])
print (lat_lng_coords[1])
'''

-----------------------------------------------------------------------------
# Adding lat,long. from csv

In [803]:
#----------------------------------Merging both dataframes on postalcodes
merged_inner = pd.merge(left=d,right=Geospatial_Coordinates, left_on='Postcode', right_on='Postal Code')
merged_inner=merged_inner.drop(columns='Postal Code')
print (merged_inner.shape)
merged_inner.head()

(103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


-----------------------------------------------------------------------------
# Mapping Scarborough and Etobicoke Data

In [804]:
scarborough_data = data_framee2[data_framee2['Borough'] == 'Scarborough'].reset_index(drop=True)
display (scarborough_data)


etobicoke_data = data_framee2[data_framee2['Borough'] == "Etobicoke"].reset_index(drop=True)
etobicoke_data


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,M1B,Scarborough,Malvern,43.806686,-79.194353
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497
3,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
4,M1C,Scarborough,Port Union,43.784535,-79.160497
5,M1E,Scarborough,Guildwood,43.763573,-79.188711
6,M1E,Scarborough,Morningside,43.763573,-79.188711
7,M1E,Scarborough,West Hill,43.763573,-79.188711
8,M1G,Scarborough,Woburn,43.770992,-79.216917
9,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M9B,Etobicoke,Cloverdale,43.650943,-79.554724
1,M9B,Etobicoke,Islington,43.650943,-79.554724
2,M9B,Etobicoke,Martin Grove,43.650943,-79.554724
3,M9B,Etobicoke,Princess Gardens,43.650943,-79.554724
4,M9B,Etobicoke,West Deane Park,43.650943,-79.554724
5,M9C,Etobicoke,Bloordale Gardens,43.643515,-79.577201
6,M9C,Etobicoke,Eringate,43.643515,-79.577201
7,M9C,Etobicoke,Markland Wood,43.643515,-79.577201
8,M9C,Etobicoke,Old Burnhamthorpe,43.643515,-79.577201
9,M9P,Etobicoke,Westmount,43.696319,-79.532242


In [805]:
from geopy.geocoders import Nominatim
address_sc = 'Scarborough, Toronto'
address_to = 'Etobicoke, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address_sc)
latitude_sc = location.latitude
longitude_sc = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_sc, longitude_sc))

location = geolocator.geocode(address_to)
latitude_to = location.latitude
longitude_to = location.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude_to, longitude_to))


The geograpical coordinate of Scarborough are 43.773077, -79.257774.
The geograpical coordinate of Etobicoke are 43.67145915, -79.5524920661167.


In [806]:
import folium
def maping(latitude,longitude,df_data,color):
    map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=10)
    for lat, lng, borough, neighborhood in zip(df_data['Latitude'], df_data['Longitude'], df_data['Borough'], df_data['Neighbourhood']):
        label = '{}, {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            parse_html=False).add_to(map_scarborough)  
    display (map_scarborough)
    
maping(latitude_sc,longitude_sc,scarborough_data,'#3186CC')
maping(latitude_to,longitude_to,etobicoke_data,'#EB1B3D')

---------------------------------
# Foursquare API
Using this API we can extract all data in JSON format and further clean it to extract venue catorory necessary for Analysis

In [807]:
CLIENT_ID = 'NRG5PBEEQPS0UJWCIWNWE3HABFW1GTHYTNLIZ3MU55PKJAZH' # your Foursquare ID
CLIENT_SECRET = 'U20WXVIWWKHEA5IS4S4LIWTPK3LZSWGW3HA3RDQ14TGQI2J3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

neighborhood_latitude = scarborough_data.loc[3, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[3, 'Longitude'] # neighborhood longitude value
neighborhood_name = scarborough_data.loc[3, 'Neighbourhood'] # neighborhood name

def getNearbyVenues(names, latitudes, longitudes, radius=500):   
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    print ('--------------------------------------------------')
    return(nearby_venues)

scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

etobicoke_venues = getNearbyVenues(names=etobicoke_data['Neighbourhood'],
                                   latitudes=etobicoke_data['Latitude'],
                                   longitudes=etobicoke_data['Longitude']
                                  )


Rouge
Malvern
Highland Creek
Rouge Hill
Port Union
Guildwood
Morningside
West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park
Ionview
Kennedy Park
Clairlea
Golden Mile
Oakridge
Cliffcrest
Cliffside
Scarborough Village West
Birch Cliff
Cliffside West
Dorset Park
Scarborough Town Centre
Wexford Heights
Maryvale
Wexford
Agincourt
Clarks Corners
Sullivan
Tam O'Shanter
Agincourt North
L'Amoreaux East
Milliken
Steeles East
L'Amoreaux West
Upper Rouge
--------------------------------------------------
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Westmount
Kingsview Village
Martin Grove Gardens
Richview Gardens
St. Phillips
Humber Bay Shores
Mimico South
New Toronto
Albion Gardens
Beaumond Heights
Humbergate
Jamestown
Mount Olive
Silverstone
South Steeles
Thistletown
Alderwood
Long Branch
Northwest
The Kingsway
Montgomery Road
Old Mill North
Humber Bay
King's Mill Park
Kingsway Park South East
Mimic

In [808]:
scarborough_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rouge,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Malvern,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
2,Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,Highland Creek,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
4,Rouge Hill,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar


In [809]:
etobicoke_venues[etobicoke_venues['Venue Category']=='Bar']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
167,Northwest,43.706748,-79.594054,Logistics Distribution,43.707554,-79.589252,Bar


### The distance from Highland creek or the bar to other places

In [826]:
l1=[]
l2=[]
from geopy.distance import great_circle
highland_creek=(43.782533,-79.163085)
for x in range(0,scarborough_venues.shape[0]):
    to_dis=(scarborough_venues['Neighborhood Latitude'].iloc[x],scarborough_venues['Neighborhood Longitude'].iloc[x])
    act_dis=great_circle(highland_creek,to_dis)
    l1.append([scarborough_venues['Neighborhood'].iloc[x]])
    l2.append(float(str(act_dis)[0:6]))
ndf=pd.DataFrame({'Neighborhood':l1,'Dis':l2})
ndf['Neighborhood']=ndf['Neighborhood'].apply(''.join)
ndf=ndf.sort_values(['Dis']).reset_index(drop=True)
ndf

try:
    scarborough_venues=scarborough_venues.drop(columns='Dis')
except:
    pass
scarborough_venues['Dis'] = l2 
scarborough_venues=scarborough_venues.sort_values(['Dis']).reset_index(drop=True)
scarborough_venues

scarborough_grouped_data = scarborough_venues.groupby('Dis').mean()
scarborough_grouped_data

,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude
Dis,,,,
0.3045,43.784535,-79.160497,43.785644,-79.162761
2.9459,43.763573,-79.188711,43.766502,-79.191117
3.6758,43.806686,-79.194353,43.807448,-79.199056
4.5085,43.770992,-79.216917,43.770559,-79.219579
6.2215,43.773136,-79.239476,43.774893,-79.240162
7.4363,43.744734,-79.239476,43.744397,-79.241896
8.0477,43.794200,-79.262029,43.792460,-79.259419
9.2806,43.757410,-79.273304,43.756825,-79.275721
9.5846,43.716316,-79.239476,43.717880,-79.240331


In [827]:
l1=[]
l2=[]
from geopy.distance import great_circle
Logistics_Distribution=(43.706748,-79.594054)
for x in range(0,etobicoke_venues.shape[0]):
    to_dis=(etobicoke_venues['Neighborhood Latitude'].iloc[x],etobicoke_venues['Neighborhood Longitude'].iloc[x])
    act_dis=great_circle(Old_Burnhamthorpe,to_dis)
    l1.append([etobicoke_venues['Neighborhood'].iloc[x]])
    l2.append(float(str(act_dis)[0:6]))
ndf=pd.DataFrame({'Neighborhood':l1,'Dis':l2})
ndf['Neighborhood']=ndf['Neighborhood'].apply(''.join)
ndf=ndf.sort_values(['Dis']).reset_index(drop=True)
ndf

try:
    etobicoke_venues=etobicoke_venues.drop(columns='Dis')
except:
    pass
etobicoke_venues['Dis'] = l2 
etobicoke_venues=etobicoke_venues.sort_values(['Dis']).reset_index(drop=True)
etobicoke_venues

etobicoke_grouped_data = etobicoke_venues.groupby('Dis').mean()
etobicoke_grouped_data

,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude
Dis,,,,
2.7451,43.643515,-79.577201,43.641653,-79.576716
4.8082,43.628841,-79.520999,43.629858,-79.519577
5.3153,43.602414,-79.543484,43.601676,-79.545335
5.3612,43.688905,-79.554724,43.690741,-79.556562
5.7641,43.653654,-79.506944,43.654276,-79.506467
6.3836,43.636258,-79.498509,43.635004,-79.496097
6.8957,43.696319,-79.532242,43.694711,-79.532493
7.1606,43.706748,-79.594054,43.707032,-79.592640
7.4185,43.605647,-79.501321,43.602080,-79.501199


### One Hot Encoding to get data ready for kmeans clustering

In [828]:
#########################################
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Dis'] = scarborough_venues['Dis'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot

,Dis,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bubble Tea Shop,Bus Line,Bus Station,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,...,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Movie Theater,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,0.3045,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.3045,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.3045,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.3045,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.3045,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0.3045,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,2.9459,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
7,2.9459,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,2.9459,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
9,2.9459,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [829]:
#########################################
etobicoke_onehot = pd.get_dummies(etobicoke_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
etobicoke_onehot['Dis'] = etobicoke_venues['Dis'] 

# move neighborhood column to the first column
fixed_columns = [etobicoke_onehot.columns[-1]] + list(etobicoke_onehot.columns[:-1])
etobicoke_onehot = etobicoke_onehot[fixed_columns]
etobicoke_onehot=etobicoke_onehot.dropna()
etobicoke_onehot

,Dis,American Restaurant,Bakery,Bar,Baseball Field,Beer Store,Burger Joint,Burrito Place,Bus Line,Café,Chinese Restaurant,Coffee Shop,Construction & Landscaping,Convenience Store,Dance Studio,Discount Store,Drugstore,Fast Food Restaurant,Flower Shop,Fried Chicken Joint,...,Hardware Store,Intersection,Liquor Store,Middle Eastern Restaurant,Mobile Phone Shop,Park,Pharmacy,Pizza Place,Pool,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Skating Rink,Smoke Shop,Social Club,Supplement Shop,Tanning Salon,Wings Joint
0,2.7451,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2.7451,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,2.7451,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2.7451,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2.7451,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2.7451,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,2.7451,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
7,2.7451,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,2.7451,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
9,2.7451,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Finding mean of Data to get frequency of neighbourhood vs venue category 

In [830]:
#################################
scarborough_grouped = scarborough_onehot.groupby('Dis').mean().reset_index()
scarborough_grouped=scarborough_grouped.sort_values(['Dis']).reset_index(drop=True)
pd.set_option('display.max_columns', 40)
scarborough_grouped

,Dis,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bubble Tea Shop,Bus Line,Bus Station,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,...,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Movie Theater,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,0.3045,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000
1,2.9459,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.142857,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.00,0.142857,0.0,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.142857,0.000000,0.00,0.0,0.000000,0.000000
2,3.6758,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000
3,4.5085,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.666667,0.00,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000
4,6.2215,0.000000,0.000000,0.142857,0.000000,0.142857,0.142857,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.142857,0.000000,0.000000,0.00,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.142857,0.000000
5,7.4363,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.00,0.0,0.000000,0.000000
6,8.0477,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.25,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.00,0.0,0.000000,0.000000
7,9.2806,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.142857,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.142857,0.000000,0.00,0.000000,...,0.142857,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.142857
8,9.5846,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000
9,10.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.166667,0.00,0.000000,0.000000,0.166667,0.166667,0.00,0.166667,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000


In [831]:
#################################
etobicoke_grouped = etobicoke_onehot.groupby('Dis').mean().reset_index()
etobicoke_grouped=etobicoke_grouped.sort_values(['Dis']).reset_index(drop=True)
pd.set_option('display.max_columns', 40)
etobicoke_grouped

,Dis,American Restaurant,Bakery,Bar,Baseball Field,Beer Store,Burger Joint,Burrito Place,Bus Line,Café,Chinese Restaurant,Coffee Shop,Construction & Landscaping,Convenience Store,Dance Studio,Discount Store,Drugstore,Fast Food Restaurant,Flower Shop,Fried Chicken Joint,...,Hardware Store,Intersection,Liquor Store,Middle Eastern Restaurant,Mobile Phone Shop,Park,Pharmacy,Pizza Place,Pool,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Skating Rink,Smoke Shop,Social Club,Supplement Shop,Tanning Salon,Wings Joint
0,2.7451,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.00,0.142857,0.000000,0.142857,0.000000,0.142857,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.142857,0.000000,0.00,0.00,0.142857,0.142857,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000
1,4.8082,0.000000,0.071429,0.000000,0.000000,0.000000,0.071429,0.071429,0.00,0.000000,0.000000,0.000000,0.000000,0.071429,0.0,0.071429,0.000000,0.071429,0.000000,0.000000,...,0.071429,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.071429,0.0,0.00,0.071429,0.071429,0.071429,0.071429
2,5.3153,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.100000,0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.100000,0.200000,0.10,0.1,0.000000,0.000000,0.00,0.100000,0.1,0.00,0.000000,0.000000,0.000000,0.000000
3,5.3612,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.25,0.25,0.000000,0.250000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000
4,5.7641,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.25,0.000000,0.000000,0.25,0.0,0.000000,0.000000,0.25,0.000000,0.0,0.25,0.000000,0.000000,0.000000,0.000000
5,6.3836,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000
6,6.8957,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.142857,0.142857,0.000000,0.000000,0.0,0.142857,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.142857,0.000000,0.142857,0.00,0.00,0.000000,0.142857,0.00,0.0,0.000000,0.000000,0.00,0.142857,0.0,0.00,0.000000,0.000000,0.000000,0.000000
7,7.1606,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.333333,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.0,0.333333,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000
8,7.4185,0.076923,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.153846,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.076923,0.076923,0.076923,...,0.000000,0.000000,0.076923,0.000000,0.00,0.00,0.076923,0.076923,0.00,0.0,0.000000,0.076923,0.00,0.076923,0.0,0.00,0.000000,0.000000,0.000000,0.000000
9,10.7010,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.142857,0.000000,0.142857,...,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.142857,0.142857,0.00,0.0,0.000000,0.000000,0.00,0.142857,0.0,0.00,0.000000,0.000000,0.000000,0.000000


In [832]:
def return_most_common_neighbourbood(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of Top Neighbourbood
columns = ['Dis']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common NHood'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common NHood'.format(ind+1))

# create a new dataframe
neighborhood1_venues_sorted = pd.DataFrame(columns=columns)
neighborhood1_venues_sorted['Dis'] = scarborough_grouped['Dis']

neighborhood2_venues_sorted = pd.DataFrame(columns=columns)
neighborhood2_venues_sorted['Dis'] = etobicoke_grouped['Dis']


for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhood1_venues_sorted.iloc[ind, 1:] = return_most_common_neighbourbood(scarborough_grouped.iloc[ind, :], num_top_venues)
display (neighborhood1_venues_sorted.head())

for ind in np.arange(etobicoke_grouped.shape[0]):
    neighborhood2_venues_sorted.iloc[ind, 1:] = return_most_common_neighbourbood(etobicoke_grouped.iloc[ind, :], num_top_venues)
(neighborhood1_venues_sorted.loc[neighborhood1_venues_sorted['Dis']=='Bar'])
#.loc[scarborough_merged['Cluster Labels'] ==x_num
scarborough_venues_grp = scarborough_venues.groupby('Dis')
scarborough_venues_grp.head()

,Dis,1st Most Common NHood,2nd Most Common NHood,3rd Most Common NHood,4th Most Common NHood,5th Most Common NHood,6th Most Common NHood,7th Most Common NHood,8th Most Common NHood,9th Most Common NHood,10th Most Common NHood
0,0.3045,History Museum,Bar,Vietnamese Restaurant,Caribbean Restaurant,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Cosmetics Shop
1,2.9459,Mexican Restaurant,Medical Center,Electronics Store,Rental Car Location,Pizza Place,Intersection,Breakfast Spot,Discount Store,Department Store,Cosmetics Shop
2,3.6758,Fast Food Restaurant,Vietnamese Restaurant,Camera Store,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Cosmetics Shop,Convenience Store,College Stadium
3,4.5085,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Cosmetics Shop
4,6.2215,Thai Restaurant,Hakka Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Bank,Caribbean Restaurant,Chinese Restaurant,Fast Food Restaurant,Electronics Store


/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Dis
0,Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar,0.3045
1,Highland Creek,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum,0.3045
2,Rouge Hill,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar,0.3045
3,Rouge Hill,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum,0.3045
4,Port Union,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar,0.3045
6,Morningside,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place,2.9459
7,West Hill,43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot,2.9459
8,Guildwood,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place,2.9459
9,Guildwood,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store,2.9459
10,Guildwood,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant,2.9459


# Clustering The data to groups

In [833]:
from sklearn.cluster import KMeans
kclusters = 3

# run k-means clustering
kmeans_sc = KMeans(n_clusters=kclusters, random_state=1,algorithm='full').fit(scarborough_grouped)
kmeans_et = KMeans(n_clusters=kclusters, random_state=1,algorithm='full').fit(etobicoke_grouped)

# check cluster labels generated for each row in the dataframe
display (kmeans_sc.labels_) 
kmeans_et.labels_ 

array([2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1], dtype=int32)

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 2], dtype=int32)

In [834]:
# add clustering labels
neighborhood1_venues_sorted.insert(0, 'Cluster Labels', kmeans_sc.labels_)

scarborough_merged = scarborough_data
scarborough_merged = pd.merge(left=scarborough_grouped_data,right=neighborhood1_venues_sorted, left_on='Dis', right_on='Dis')
scarborough_merged=scarborough_merged.dropna()
scarborough_merged['Cluster Labels'] = scarborough_merged['Cluster Labels'].apply(np.int64)
scarborough_merged # check the last columns!

,Dis,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Cluster Labels,1st Most Common NHood,2nd Most Common NHood,3rd Most Common NHood,4th Most Common NHood,5th Most Common NHood,6th Most Common NHood,7th Most Common NHood,8th Most Common NHood,9th Most Common NHood,10th Most Common NHood
0,0.3045,43.784535,-79.160497,43.785644,-79.162761,2,History Museum,Bar,Vietnamese Restaurant,Caribbean Restaurant,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Cosmetics Shop
1,2.9459,43.763573,-79.188711,43.766502,-79.191117,2,Mexican Restaurant,Medical Center,Electronics Store,Rental Car Location,Pizza Place,Intersection,Breakfast Spot,Discount Store,Department Store,Cosmetics Shop
2,3.6758,43.806686,-79.194353,43.807448,-79.199056,2,Fast Food Restaurant,Vietnamese Restaurant,Camera Store,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Cosmetics Shop,Convenience Store,College Stadium
3,4.5085,43.770992,-79.216917,43.770559,-79.219579,2,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Cosmetics Shop
4,6.2215,43.773136,-79.239476,43.774893,-79.240162,0,Thai Restaurant,Hakka Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Bank,Caribbean Restaurant,Chinese Restaurant,Fast Food Restaurant,Electronics Store
5,7.4363,43.744734,-79.239476,43.744397,-79.241896,0,Playground,Cosmetics Shop,Vietnamese Restaurant,Camera Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store
6,8.0477,43.794200,-79.262029,43.792460,-79.259419,0,Sandwich Place,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Caribbean Restaurant,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
7,9.2806,43.757410,-79.273304,43.756825,-79.275721,0,Indian Restaurant,Vietnamese Restaurant,Pet Store,Brewery,Light Rail Station,Chinese Restaurant,Caribbean Restaurant,Fast Food Restaurant,Electronics Store,Discount Store
8,9.5846,43.716316,-79.239476,43.717880,-79.240331,0,American Restaurant,Movie Theater,Motel,Café,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Cosmetics Shop,Convenience Store
9,10.0000,43.727929,-79.262029,43.728070,-79.265922,0,Discount Store,Department Store,Convenience Store,Coffee Shop,Chinese Restaurant,Bus Station,Vietnamese Restaurant,Camera Store,Fried Chicken Joint,Fast Food Restaurant


In [835]:
# add clustering labels
neighborhood2_venues_sorted.insert(0, 'Cluster Labels', kmeans_et.labels_)

etobicoke_merged = etobicoke_data
etobicoke_merged = pd.merge(left=etobicoke_grouped_data,right=neighborhood2_venues_sorted, left_on='Dis', right_on='Dis')
etobicoke_merged=etobicoke_merged.dropna()
etobicoke_merged['Cluster Labels'] = etobicoke_merged['Cluster Labels'].apply(np.int64)
etobicoke_merged # check the last columns!

,Dis,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Cluster Labels,1st Most Common NHood,2nd Most Common NHood,3rd Most Common NHood,4th Most Common NHood,5th Most Common NHood,6th Most Common NHood,7th Most Common NHood,8th Most Common NHood,9th Most Common NHood,10th Most Common NHood
0,2.7451,43.643515,-79.577201,43.641653,-79.576716,1,Coffee Shop,Pizza Place,Beer Store,Liquor Store,Convenience Store,Café,Pharmacy,Fast Food Restaurant,Drugstore,Discount Store
1,4.8082,43.628841,-79.520999,43.629858,-79.519577,1,Wings Joint,Hardware Store,Bakery,Burger Joint,Burrito Place,Convenience Store,Discount Store,Fast Food Restaurant,Grocery Store,Tanning Salon
2,5.3153,43.602414,-79.543484,43.601676,-79.545335,1,Pizza Place,Gym,Pub,Dance Studio,Coffee Shop,Pharmacy,Pool,Sandwich Place,Skating Rink,Beer Store
3,5.3612,43.688905,-79.554724,43.690741,-79.556562,1,Pizza Place,Mobile Phone Shop,Bus Line,Park,Wings Joint,Fast Food Restaurant,Drugstore,Discount Store,Dance Studio,Convenience Store
4,5.7641,43.653654,-79.506944,43.654276,-79.506467,1,Pool,Smoke Shop,River,Park,Café,Drugstore,Discount Store,Dance Studio,Convenience Store,Construction & Landscaping
5,6.3836,43.636258,-79.498509,43.635004,-79.496097,0,Construction & Landscaping,Baseball Field,Wings Joint,Coffee Shop,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store,Dance Studio,Convenience Store
6,6.8957,43.696319,-79.532242,43.694711,-79.532493,0,Chinese Restaurant,Pizza Place,Discount Store,Intersection,Sandwich Place,Middle Eastern Restaurant,Coffee Shop,Drugstore,Dance Studio,Convenience Store
7,7.1606,43.706748,-79.594054,43.707032,-79.592640,0,Bar,Drugstore,Rental Car Location,Wings Joint,Coffee Shop,Flower Shop,Fast Food Restaurant,Discount Store,Dance Studio,Convenience Store
8,7.4185,43.605647,-79.501321,43.602080,-79.501199,0,Café,Gym,Pizza Place,Bakery,Fast Food Restaurant,Flower Shop,Fried Chicken Joint,Liquor Store,Pharmacy,American Restaurant
9,10.7010,43.739416,-79.588437,43.741718,-79.585192,2,Fried Chicken Joint,Pharmacy,Grocery Store,Fast Food Restaurant,Beer Store,Sandwich Place,Pizza Place,Drugstore,Discount Store,Dance Studio


### Mapping the Cluster

In [847]:
import matplotlib.cm as cm
import matplotlib.colors as colors

def map_clust(latitude,longitude,dff):
    # create map
    map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

    # set color scheme for the clusters
    x = np.arange(kclusters)
    ys = [i + x + (i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map
    markers_colors = []
    for lat, lon, poi, cluster in zip(dff['Neighborhood Latitude'], dff['Neighborhood Longitude'], dff['1st Most Common NHood'], dff['Cluster Labels']):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.9).add_to(map_clusters)

    display (map_clusters)
map_clust(latitude_sc,longitude_sc,scarborough_merged)
map_clust(latitude_to,longitude_to,etobicoke_merged)

# Analysing every cluster individually

In [837]:
for x_num in range(0,kclusters):
    scarborough_merged.loc[scarborough_merged['Cluster Labels'] ==x_num, scarborough_merged.columns[[1]+ [2] + list(range(5, scarborough_merged.shape[1]))]]
    xc = scarborough_merged.loc[scarborough_merged['Cluster Labels'] ==x_num]
    import folium
    map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=10)

    for lat, lng, borough, neighborhood in zip(xc['Neighborhood Latitude'], xc['Neighborhood Longitude'], xc['1st Most Common NHood'], xc['Dis']):
        label = '{}'.format(borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#1478dy',
            fill_opacity=0.7,
            parse_html=False).add_to(map_scarborough) 
    display (xc) 
    #print (scarborough_merged.loc[scarborough_merged['Cluster Labels'] ==0, scarborough_merged.columns[[1]+ [2] + list(range(5, scarborough_merged.shape[1]))]])
    display (map_scarborough)
    print ('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')

,Dis,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Cluster Labels,1st Most Common NHood,2nd Most Common NHood,3rd Most Common NHood,4th Most Common NHood,5th Most Common NHood,6th Most Common NHood,7th Most Common NHood,8th Most Common NHood,9th Most Common NHood,10th Most Common NHood
4,6.2215,43.773136,-79.239476,43.774893,-79.240162,0,Thai Restaurant,Hakka Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Bank,Caribbean Restaurant,Chinese Restaurant,Fast Food Restaurant,Electronics Store
5,7.4363,43.744734,-79.239476,43.744397,-79.241896,0,Playground,Cosmetics Shop,Vietnamese Restaurant,Camera Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store
6,8.0477,43.794200,-79.262029,43.792460,-79.259419,0,Sandwich Place,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Caribbean Restaurant,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
7,9.2806,43.757410,-79.273304,43.756825,-79.275721,0,Indian Restaurant,Vietnamese Restaurant,Pet Store,Brewery,Light Rail Station,Chinese Restaurant,Caribbean Restaurant,Fast Food Restaurant,Electronics Store,Discount Store
8,9.5846,43.716316,-79.239476,43.717880,-79.240331,0,American Restaurant,Movie Theater,Motel,Café,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Cosmetics Shop,Convenience Store
9,10.0000,43.727929,-79.262029,43.728070,-79.265922,0,Discount Store,Department Store,Convenience Store,Coffee Shop,Chinese Restaurant,Bus Station,Vietnamese Restaurant,Camera Store,Fried Chicken Joint,Fast Food Restaurant


xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


,Dis,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Cluster Labels,1st Most Common NHood,2nd Most Common NHood,3rd Most Common NHood,4th Most Common NHood,5th Most Common NHood,6th Most Common NHood,7th Most Common NHood,8th Most Common NHood,9th Most Common NHood,10th Most Common NHood
10,10.407,43.815252,-79.284577,43.815199,-79.289821,1,Asian Restaurant,Playground,Park,Vietnamese Restaurant,Camera Store,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Cosmetics Shop
11,11.255,43.750071,-79.295849,43.746917,-79.295422,1,Middle Eastern Restaurant,Sandwich Place,Breakfast Spot,Auto Garage,Vietnamese Restaurant,Athletics & Sports,Caribbean Restaurant,Fast Food Restaurant,Electronics Store,Discount Store
12,11.337,43.781637,-79.304302,43.780247,-79.301393,1,Pharmacy,Pizza Place,Bank,Fried Chicken Joint,Italian Restaurant,Noodle House,Fast Food Restaurant,Thai Restaurant,Chinese Restaurant,Department Store
13,12.582,43.711112,-79.284577,43.711625,-79.281463,1,Bakery,Bus Line,Bus Station,Intersection,Metro Station,Park,Fast Food Restaurant,Soccer Field,Bank,Bar
14,12.608,43.799525,-79.318389,43.798574,-79.318554,1,Fast Food Restaurant,Chinese Restaurant,Gym,Breakfast Spot,Electronics Store,Coffee Shop,Camera Store,Bubble Tea Shop,Grocery Store,Sandwich Place
15,12.911,43.692657,-79.264848,43.694203,-79.262554,1,Café,Skating Rink,College Stadium,General Entertainment,Athletics & Sports,Auto Garage,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


,Dis,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Cluster Labels,1st Most Common NHood,2nd Most Common NHood,3rd Most Common NHood,4th Most Common NHood,5th Most Common NHood,6th Most Common NHood,7th Most Common NHood,8th Most Common NHood,9th Most Common NHood,10th Most Common NHood
0,0.3045,43.784535,-79.160497,43.785644,-79.162761,2,History Museum,Bar,Vietnamese Restaurant,Caribbean Restaurant,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Cosmetics Shop
1,2.9459,43.763573,-79.188711,43.766502,-79.191117,2,Mexican Restaurant,Medical Center,Electronics Store,Rental Car Location,Pizza Place,Intersection,Breakfast Spot,Discount Store,Department Store,Cosmetics Shop
2,3.6758,43.806686,-79.194353,43.807448,-79.199056,2,Fast Food Restaurant,Vietnamese Restaurant,Camera Store,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Cosmetics Shop,Convenience Store,College Stadium
3,4.5085,43.770992,-79.216917,43.770559,-79.219579,2,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Cosmetics Shop


xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


In [822]:
for x_num in range(0,kclusters):
    etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] ==x_num, etobicoke_merged.columns[[1]+ [2] + list(range(5, etobicoke_merged.shape[1]))]]
    xc = etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] ==x_num]
    map_etobicoke = folium.Map(location=[latitude_to, longitude_sc], zoom_start=10)

    for lat, lng, borough, neighborhood in zip(xc['Neighborhood Latitude'], xc['Neighborhood Longitude'], xc['1st Most Common NHood'], xc['Dis']):
        label = '{}'.format(borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#1478dy',
            fill_opacity=0.7,
            parse_html=False).add_to(map_etobicoke) 
    display (xc) 
    #print (scarborough_merged.loc[scarborough_merged['Cluster Labels'] ==0, scarborough_merged.columns[[1]+ [2] + list(range(5, scarborough_merged.shape[1]))]])
    display (map_etobicoke)
    print ('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')

,Dis,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Cluster Labels,1st Most Common NHood,2nd Most Common NHood,3rd Most Common NHood,4th Most Common NHood,5th Most Common NHood,6th Most Common NHood,7th Most Common NHood,8th Most Common NHood,9th Most Common NHood,10th Most Common NHood
5,6.3836,43.636258,-79.498509,43.635004,-79.496097,0,Construction & Landscaping,Baseball Field,Wings Joint,Coffee Shop,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store,Dance Studio,Convenience Store
6,6.8957,43.696319,-79.532242,43.694711,-79.532493,0,Chinese Restaurant,Pizza Place,Discount Store,Intersection,Sandwich Place,Middle Eastern Restaurant,Coffee Shop,Drugstore,Dance Studio,Convenience Store
7,7.1606,43.706748,-79.594054,43.707032,-79.592640,0,Bar,Drugstore,Rental Car Location,Wings Joint,Coffee Shop,Flower Shop,Fast Food Restaurant,Discount Store,Dance Studio,Convenience Store
8,7.4185,43.605647,-79.501321,43.602080,-79.501199,0,Café,Gym,Pizza Place,Bakery,Fast Food Restaurant,Flower Shop,Fried Chicken Joint,Liquor Store,Pharmacy,American Restaurant


xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


,Dis,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Cluster Labels,1st Most Common NHood,2nd Most Common NHood,3rd Most Common NHood,4th Most Common NHood,5th Most Common NHood,6th Most Common NHood,7th Most Common NHood,8th Most Common NHood,9th Most Common NHood,10th Most Common NHood
0,2.7451,43.643515,-79.577201,43.641653,-79.576716,1,Coffee Shop,Pizza Place,Beer Store,Liquor Store,Convenience Store,Café,Pharmacy,Fast Food Restaurant,Drugstore,Discount Store
1,4.8082,43.628841,-79.520999,43.629858,-79.519577,1,Wings Joint,Hardware Store,Bakery,Burger Joint,Burrito Place,Convenience Store,Discount Store,Fast Food Restaurant,Grocery Store,Tanning Salon
2,5.3153,43.602414,-79.543484,43.601676,-79.545335,1,Pizza Place,Gym,Pub,Dance Studio,Coffee Shop,Pharmacy,Pool,Sandwich Place,Skating Rink,Beer Store
3,5.3612,43.688905,-79.554724,43.690741,-79.556562,1,Pizza Place,Mobile Phone Shop,Bus Line,Park,Wings Joint,Fast Food Restaurant,Drugstore,Discount Store,Dance Studio,Convenience Store
4,5.7641,43.653654,-79.506944,43.654276,-79.506467,1,Pool,Smoke Shop,River,Park,Café,Drugstore,Discount Store,Dance Studio,Convenience Store,Construction & Landscaping


xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


,Dis,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Cluster Labels,1st Most Common NHood,2nd Most Common NHood,3rd Most Common NHood,4th Most Common NHood,5th Most Common NHood,6th Most Common NHood,7th Most Common NHood,8th Most Common NHood,9th Most Common NHood,10th Most Common NHood
9,10.701,43.739416,-79.588437,43.741718,-79.585192,2,Fried Chicken Joint,Pharmacy,Grocery Store,Fast Food Restaurant,Beer Store,Sandwich Place,Pizza Place,Drugstore,Discount Store,Dance Studio


xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


### The cluster are perfetly divided as we can see 1st most common venue is mostly seperated in each cluster.
### Every cluster has its own venue category and if we have to further divide it, it would have done even better

In [823]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] ==0, scarborough_merged.columns[[1]+ [2] + list(range(5, scarborough_merged.shape[1]))]]
xc = scarborough_merged.loc[scarborough_merged['Cluster Labels'] ==0]
cluster1=xc
scarborough_merged.loc[scarborough_merged['Cluster Labels'] ==1, scarborough_merged.columns[[1]+ [2] + list(range(5, scarborough_merged.shape[1]))]]
xc = scarborough_merged.loc[scarborough_merged['Cluster Labels'] ==1]
cluster2=xc
scarborough_merged.loc[scarborough_merged['Cluster Labels'] ==2, scarborough_merged.columns[[1]+ [2] + list(range(5, scarborough_merged.shape[1]))]]
xc = scarborough_merged.loc[scarborough_merged['Cluster Labels'] ==2]
cluster3=xc

etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] ==0, etobicoke_merged.columns[[1]+ [2] + list(range(5, etobicoke_merged.shape[1]))]]
xc = etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] ==0]
cluster4=xc
etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] ==1, etobicoke_merged.columns[[1]+ [2] + list(range(5, etobicoke_merged.shape[1]))]]
xc = etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] ==1]
cluster5=xc
etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] ==2, etobicoke_merged.columns[[1]+ [2] + list(range(5, etobicoke_merged.shape[1]))]]
xc = etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] ==2]
cluster6=xc

# Finding Similarity between clusters

In [841]:
cluster1_onehot = pd.get_dummies(cluster1, prefix="", prefix_sep="")
cluster1_onehot=np.asarray(cluster1_onehot).flatten()
c1 = (cluster1_onehot.mean())

cluster2_onehot = pd.get_dummies(cluster2, prefix="", prefix_sep="")
cluster2_onehot=np.array(cluster2_onehot).flatten()
c2 = (cluster2_onehot.mean())
#cluster2_onehot=cluster2_onehot.reshape(1,cluster2_onehot.shape[0])

cluster3_onehot = pd.get_dummies(cluster3, prefix="", prefix_sep="")
cluster3_onehot=np.array(cluster3_onehot).flatten()
c3 = (cluster3_onehot.mean())

cluster4_onehot = pd.get_dummies(cluster4, prefix="", prefix_sep="")
cluster4_onehot=np.asarray(cluster4_onehot).flatten()
c4 = (cluster4_onehot.mean())

cluster5_onehot = pd.get_dummies(cluster5, prefix="", prefix_sep="")
cluster5_onehot=np.array(cluster5_onehot).flatten()
c5 = (cluster5_onehot.mean())

cluster6_onehot = pd.get_dummies(cluster6, prefix="", prefix_sep="")
cluster6_onehot=np.array(cluster6_onehot).flatten()
c6 = (cluster6_onehot.mean())

In [848]:
import scipy.spatial.distance as distance
print ('Similarity in Bar cluster and {} Cluster is: '.format('Cluster 1')+str(1-abs(c3-c1)))
print ('Similarity in Bar cluster and {} Cluster is: '.format('Cluster 2')+str(1-abs(c3-c2)))
print ('Similarity in Bar cluster and {} Cluster is: '.format('Cluster 4')+str(1-abs(c4-c5)))
print ('Similarity in Bar cluster and {} Cluster is: '.format('Cluster 5')+str(abs(1-abs(c4-c6))))

Similarity in Bar cluster and Cluster 1 Cluster is: 0.2669983146806716
Similarity in Bar cluster and Cluster 2 Cluster is: 0.22324011150240164
Similarity in Bar cluster and Cluster 4 Cluster is: 0.7074259441413882
Similarity in Bar cluster and Cluster 5 Cluster is: 0.692553652002758


# As the conclusion, there are no bars in either of cluster so finally after finding similarity we can say its more prefrable to set up a bar in cluster two